# Tarea 1: DS Lab
- Ricardo Arancibia
- José Díaz
- Javier Santibáñez

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## P1

In [2]:
# 1
df_all = pd.read_csv('data/raw/w{}/metrocuadrado_all_w{}.csv'.format(str(13),str(13)))
for NN in range(14,18):
    direc = 'data/raw/w{}/metrocuadrado_all_w{}.csv'.format(str(NN),str(NN))
    df_aux = pd.read_csv(direc)
    df_all = pd.concat([df_all,df_aux], ignore_index= True)
df_all['Archivo'] = 'all' 
df_all.drop_duplicates(inplace=True, ignore_index= True)
print(df_all.shape)

(16295, 9)


In [3]:
df_fur = pd.read_csv('data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(str(13),str(13)))
for NN in range(14,18):
    direc = 'data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(str(NN),str(NN))
    df_aux = pd.read_csv(direc)
    df_fur = pd.concat([df_fur,df_aux], ignore_index= True)
df_fur['Archivo'] = 'furnished'
df_fur.drop_duplicates(inplace=True, ignore_index= True)
print(df_fur.shape)

(2099, 9)


In [9]:
df = pd.concat([df_all,df_fur],ignore_index= True)
print(df.shape)
df.head()

(18394, 9)


,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,all
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,all


In [10]:
# 2
df.dropna(subset=['price','n_rooms','n_bath','surface'], inplace=True)
df.reset_index(drop=True,inplace=True)
print(df.shape)

(18143, 9)


In [11]:
# 2.a
df['price'] = [x.replace('$','') for x in df['price']]
df['price'] = [x.replace('.','') for x in df['price']]
df['surface'] = [x.replace(' m2','') for x in df['surface']]
df['price'] = df['price'].astype('float64')
df['surface'] = df['surface'].astype('float64')
for i in range(len(df)):
    x = df['n_bath'][i]
    y = df['n_rooms'][i]
    if type(x)== float:
        df['n_bath'][i] = str(int(x)) # 3.0 <-> '3'
    if type(y)== float:
        df['n_rooms'][i] = str(int(y))

In [12]:
# 2.b
df_aux = df['property_type|rent_type|location'].str.split(pat=',',expand =True)
df_aux[0] = [x.replace(' ',', ', 1) for x in df_aux[0]]
df_aux = pd.concat([df_aux[0].str.split(pat=',',expand =True), df_aux[1]], axis=1, ignore_index=True)
df_aux.columns = ['property_type','rent_type','location']
df_aux['rent_type'] = [x.replace('en ','') for x in df_aux['rent_type']]
df = pd.concat([df_aux,df.drop(columns=['property_type|rent_type|location'])], axis=1)
df['location'] = df['location'].apply(str.lower)
df['location'] = [x.replace(' bogotá d.c..','') for x in df['location']]
df['location'] = df['location'].apply(str.strip)
df.head(3)

,property_type,rent_type,location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,Casa,Arriendo,la soledad norte,1050000.0,3,2,63.0,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all
1,Casa,Arriendo,ciudad salitre sur-oriental,3930000.0,3,2,100.0,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all
2,Casa,Arriendo,villa de los alpes ii sector,750000.0,3,1,90.0,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all


In [13]:
# 3
df['price_by_m2'] = df['price']/df['surface']

lista = []
for x in df['url']:
    if x.find('-garajes')==-1:
        lista.append('0')
    else:
        lista.append(x[x.find('banos-')+6: x.find('-garajes')])
df['n_garajes'] = lista

In [14]:
# 4
df['tipo_producto'] = np.zeros(len(df)) # categoria 0
indices=df.query('80<=surface <120 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 1
indices=df.query('120<=surface <180 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 2
indices=df.query('180<=surface <240 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 3
indices=df.query('240<=surface <360 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 4
indices=df.query('360<=surface <=460 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 5
indices=df.query('40<=surface <60 and property_type=="Apartamento"').index
df.loc[indices,'tipo_producto'] = 6
indices=df.query('60<=surface <80 and property_type=="Apartamento"').index
df.loc[indices,'tipo_producto'] = 7
indices=df.query('80<=surface <=120 and property_type=="Apartamento"').index
df.loc[indices,'tipo_producto'] = 8

In [15]:
# 5
df_upz = pd.read_csv('data/asignacion_upz/barrio-upz.csv')
df_upz.rename(columns={'pro_location':'location'}, inplace=True)
df = pd.merge(df, df_upz, how='left', on='location')
df.rename(columns={'UPlCodigo':'UPZ_Codigo'}, inplace=True)
print('Numero de observaciones a los que no se les puede adjuntar un codigo UPZ: ', df['UPZ_Codigo'].isna().sum())
print("Porcentaje asignable: ", round((len(df)-df['UPZ_Codigo'].isna().sum())/len(df)*100,1))
df.dropna(subset=['UPZ_Codigo'], inplace=True)
df.reset_index(drop=True,inplace=True)

Numero de observaciones a los que no se les puede adjuntar un codigo UPZ:  2031
Porcentaje asignable:  88.8


In [16]:
# 6
df_est_pobl = pd.read_csv('data/estadisticas_upz/estadisticas_poblacion.csv',index_col=0)
df_est_pobl.rename(columns={'upz':'UPZ_Codigo'}, inplace=True)
df = pd.merge(df, df_est_pobl, how='left', on='UPZ_Codigo')

df_est_ind = pd.read_csv('data/estadisticas_upz/indice_inseguridad.csv',index_col=0)
df_est_ind.rename(columns={'UPlCodigo':'UPZ_Codigo'}, inplace=True)
df = pd.merge(df, df_est_ind, how='left', on='UPZ_Codigo')

df_est_por = pd.read_csv('data/estadisticas_upz/porcentaje_areas_verdes.csv',index_col=0)
df_est_por.rename(columns={'cod_upz':'UPZ_Codigo'}, inplace=True)
df_est_por['UPZ_Codigo'] = df_est_por['UPZ_Codigo'].astype(int)
df_est_por['UPZ_Codigo'] = ['UPZ'+str(x) for x in df_est_por['UPZ_Codigo']]
df = pd.merge(df, df_est_por, how='left', on='UPZ_Codigo')

df['densidad_pobl'] = df['personas']/df['UPlArea']

In [ ]:
df.head()